In [20]:
import pandas as pd
import pickle

In [21]:
file_path = "/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/cubert_embeddings/df_dc_embeded_by_line_cubert.pkl"
with open(file_path, "rb") as fh:
  df = pickle.load(fh)

/var/folders/d8/cr798cb57kzd5kz_r8g8jm380000gn/T/ipykernel_31878/2678406953.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df = pickle.load(fh)


In [22]:
df.head()

,sample_id,severity,method,label,embeded_sequence_sum,embeded_sequence_avg
0,8077,critical,public class DruidPooledCallableStatement exte...,1,"[[493.20623779296875, -94.64544677734375, -291...","[[0.5314722657203674, -0.10198862850666046, -0..."
1,5553,critical,public interface IAopReferenceModel {\n\t\n\tv...,1,"[[7.537168979644775, -14.017938613891602, -10....","[[0.41873160004615784, -0.7787743806838989, -0..."
2,9341,critical,@XmlAccessorType(XmlAccessType.FIELD)\n@XmlTyp...,1,"[[0.23157545924186707, -9.463057518005371, -18...","[[0.006258795969188213, -0.2557583153247833, -..."
3,10419,critical,public class Ipam {\n private String driver;\...,1,"[[37.18507385253906, -24.23702049255371, -40.5...","[[0.5468392968177795, -0.35642677545547485, -0..."
4,12232,critical,public class DruidPooledCallableStatement exte...,1,"[[493.20623779296875, -94.64544677734375, -291...","[[0.5314722657203674, -0.10198862850666046, -0..."


In [23]:
embeded_sum = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_sum']]
embeded_avg = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_avg']]
embeded_sum.head()

,sample_id,severity,label,embeded_sequence_sum
0,8077,critical,1,"[[493.20623779296875, -94.64544677734375, -291..."
1,5553,critical,1,"[[7.537168979644775, -14.017938613891602, -10...."
2,9341,critical,1,"[[0.23157545924186707, -9.463057518005371, -18..."
3,10419,critical,1,"[[37.18507385253906, -24.23702049255371, -40.5..."
4,12232,critical,1,"[[493.20623779296875, -94.64544677734375, -291..."


In [24]:
print(embeded_sum["embeded_sequence_sum"].head())
print(embeded_sum["embeded_sequence_sum"].apply(type).value_counts())

0    [[493.20623779296875, -94.64544677734375, -291...
1    [[7.537168979644775, -14.017938613891602, -10....
2    [[0.23157545924186707, -9.463057518005371, -18...
3    [[37.18507385253906, -24.23702049255371, -40.5...
4    [[493.20623779296875, -94.64544677734375, -291...
Name: embeded_sequence_sum, dtype: object
embeded_sequence_sum
<class 'list'>    3694
Name: count, dtype: int64


In [25]:
def process_embedding(x):
    if x is None:
        return []  
    elif isinstance(x, pd.Series):
        return " ".join(map(str, x))
    else:
        return str(x)  

embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(process_embedding)


In [26]:
print(embeded_sum["embeded_sequence_sum"].head())

0    [[493.20623779296875, -94.64544677734375, -291...
1    [[7.537168979644775, -14.017938613891602, -10....
2    [[0.23157545924186707, -9.463057518005371, -18...
3    [[37.18507385253906, -24.23702049255371, -40.5...
4    [[493.20623779296875, -94.64544677734375, -291...
Name: embeded_sequence_sum, dtype: object


In [27]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [28]:
import re

def parse_flat_embedding(embedding_str):
    # Ensure input is a string
    if not isinstance(embedding_str, str):
        return []
    try:
        # Extract numerical values from the string
        values = re.findall(r"[-+]?\d*\.\d+|\d+", embedding_str)
        return [float(v) for v in values]
    except Exception as e:
        print(f"Error parsing: {embedding_str} -> {e}")
        return []


In [29]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(parse_flat_embedding)

In [30]:
n = len(embeded_sum["embeded_sequence_sum"][0])
print(f"Embedding length: {n}")

embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")


Embedding length: 768
Most common sequence length: 768
Count of rows with this length: 3681


In [31]:
 # Function to handle the trimming or padding of the sequence
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * n
)

print(embeded_sum["embeded_sequence_sum"].head())

0    [493.20623779296875, -94.64544677734375, -291....
1    [7.537168979644775, -14.017938613891602, -10.7...
2    [0.23157545924186707, -9.463057518005371, -18....
3    [37.18507385253906, -24.23702049255371, -40.55...
4    [493.20623779296875, -94.64544677734375, -291....
Name: embeded_sequence_sum, dtype: object


In [32]:
embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
print(embed_lengths.value_counts())


embeded_sequence_sum
768    3694
Name: count, dtype: int64


In [33]:
# Create column names for embeddings
columns = [f'emb_{i+1}' for i in range(n)]

# Create a DataFrame for embedding values
data = pd.DataFrame(embeded_sum["embeded_sequence_sum"].to_list(), columns=columns)

# Add metadata columns
data['label'] = embeded_sum['label']
data['sample_id'] = embeded_sum['sample_id']
data['severity'] = embeded_sum['severity']

# Display the result
print(data.head())


        emb_1      emb_2       emb_3       emb_4      emb_5       emb_6  \
0  493.206238 -94.645447 -291.701904  601.198914 -32.923615 -662.838440   
1    7.537169 -14.017939  -10.766094   16.649782  -1.662095  -11.386008   
2    0.231575  -9.463058  -18.066072   34.561588  -7.121534  -28.618713   
3   37.185074 -24.237020  -40.553955   42.658749  -3.337812  -17.803162   
4  493.206238 -94.645447 -291.701904  601.198914 -32.923615 -662.838440   

        emb_7       emb_8       emb_9      emb_10  ...     emb_762  \
0  809.795105 -380.377197  825.971680  420.571716  ...  194.174377   
1   16.042494   -3.683760   25.835314    1.784303  ...   -8.649843   
2   19.281786   -5.490053   35.314980   12.210077  ...  -12.267640   
3   60.351562  -24.570055   50.746403   32.497662  ...    6.340598   
4  809.795105 -380.377197  825.971680  420.571716  ...  194.174377   

      emb_763     emb_764     emb_765     emb_766      emb_767     emb_768  \
0 -267.659790 -500.838837  256.607788  408.753296 

In [34]:
dataset_path =  "/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/cubert_embeddings/processed_embeddings/cubert_embed_line_by_line_sum.pkl"
pd.to_pickle(data, dataset_path)

In [35]:
print(embeded_avg["embeded_sequence_avg"].head())
print(embeded_avg["embeded_sequence_avg"].apply(type).value_counts())

0    [[0.5314722657203674, -0.10198862850666046, -0...
1    [[0.41873160004615784, -0.7787743806838989, -0...
2    [[0.006258795969188213, -0.2557583153247833, -...
3    [[0.5468392968177795, -0.35642677545547485, -0...
4    [[0.5314722657203674, -0.10198862850666046, -0...
Name: embeded_sequence_avg, dtype: object
embeded_sequence_avg
<class 'list'>    3694
Name: count, dtype: int64


In [36]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [37]:
n = len(embeded_avg["embeded_sequence_avg"][0])
print(f"Embedding length: {n}")

Embedding length: 16037


In [38]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(parse_flat_embedding)

In [39]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")

Most common sequence length: 768
Count of rows with this length: 3336


In [40]:
# Function to handle the trimming or padding of the sequence
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * n
)

print(embeded_avg["embeded_sequence_avg"].head())

0    [0.5314722657203674, -0.10198862850666046, -0....
1    [0.41873160004615784, -0.7787743806838989, -0....
2    [0.006258795969188213, -0.2557583153247833, -0...
3    [0.5468392968177795, -0.35642677545547485, -0....
4    [0.5314722657203674, -0.10198862850666046, -0....
Name: embeded_sequence_avg, dtype: object


In [41]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
print(embed_lengths.value_counts())

embeded_sequence_avg
768    3694
Name: count, dtype: int64


In [42]:
columns = [f'emb_{i+1}' for i in range(n)]
data = pd.DataFrame(embeded_avg["embeded_sequence_avg"].to_list(), columns=columns)
data['label'] = embeded_avg['label']
data['sample_id'] = embeded_avg['sample_id']
data['severity'] = embeded_avg['severity']
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,0.531472,-0.101989,-0.314334,0.647844,-0.035478,-0.714266,0.872624,-0.409889,0.890056,0.453202,...,0.209240,-0.288426,-0.539697,0.276517,0.440467,1.152947,-0.153769,1,8077,critical
1,0.418732,-0.778774,-0.598116,0.924988,-0.092339,-0.632556,0.891250,-0.204653,1.435295,0.099128,...,-0.480547,-0.364436,-1.308979,1.307274,-0.088777,1.309955,-0.166052,1,5553,critical
2,0.006259,-0.255758,-0.488272,0.934097,-0.192474,-0.773479,0.521129,-0.148380,0.954459,0.330002,...,0.690031,-0.331558,-0.138192,-0.551222,0.302782,-0.032620,0.666646,1,9341,critical
3,0.546839,-0.356427,-0.596382,0.627335,-0.049085,-0.261811,0.887523,-0.361324,0.746271,0.477907,...,0.093244,0.153900,-0.112701,0.416357,0.170588,0.590961,-0.729293,1,10419,critical
4,0.531472,-0.101989,-0.314334,0.647844,-0.035478,-0.714266,0.872624,-0.409889,0.890056,0.453202,...,0.209240,-0.288426,-0.539697,0.276517,0.440467,1.152947,-0.153769,1,12232,critical


In [43]:
dataset_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/data_class/cubert_embeddings/processed_embeddings/cubert_embed_line_by_line_avg.pkl'
pd.to_pickle(data, dataset_path)